Инициализация 1d-сетки

In [ ]:
import numpy as np
import opticaldevicelib_1d as od_1d
od_1d.OpticalDevice.init_values(new_dx=5e-10, new_Nx=2**20, gpu_use=True)
print(f"границы поля X +- {od_1d.OpticalDevice.Nx * od_1d.OpticalDevice.dx * 1e6} мкм")

Инициализация линзы для 1d-сетки

In [ ]:
p = od_1d.PointSource(z=100, En=12) 
E_ps1 = p.E() / p.E() # плоская волна

N_gl = 40
Copy_flag = True
arr_len = 2 if Copy_flag else 2*N_gl

phases = 2*np.pi*np.random.rand(arr_len)
w_s = (np.random.rand(arr_len)-0.5)*np.pi/180*0
s_s  = (np.random.rand(arr_len)-0.5)*2e-6*0


crl = od_1d.CRL(lam=p.lam, arr_start=E_ps1, z=0,\
                 R=6.25e-6, A=50e-6, d=2e-6, N_lens=N_gl, mol="Si", dens=2.33)

crlm = od_1d.CRLm(lam=p.lam, arr_start=E_ps1, z=0, \
                   R=6.25e-6, A=50e-6, d=2e-6, N_lens=N_gl, mol="Si", dens=2.33, \
                   copy=Copy_flag, arr_phase=phases, arr_s=s_s, arr_w=w_s, b=-1e-6, m=3e8)

# crlm = crl
focus = crlm.focus()
crlm.set_z(z=focus)
print(f"фокусное расстояние = {focus * 1e3 :.3f} мм")
print(focus)
print(f"оценка макс. числа линз = {int(crlm.Lc() * np.pi / 2)}")

Построение профиля поверхности элемента (при необходимости)

In [ ]:
# %matplotlib widget
import matplotlib.pyplot as plt
T_arr = crlm.T().get()
x_arr = crlm.x.get()
plt.plot(x_arr*1e6, crl.T().get()*1e6, label="ideal")
plt.plot(x_arr*1e6, T_arr*1e6, label="1 μm")
plt.xlim(-50, 50)
plt.grid()
# plt.legend()
plt.xlabel("$x$, μm")
plt.ylabel("$z$, μm")
plt.savefig("shape_elem", dpi=1200);
plt.show();

In [ ]:
I_distr_max, I_max, dz_max, dx_max, fwhm, focus_image, z_arr, div_ang, x_cutted = crlm.focus_params(n_dots=1000, eps=3e-3, n_cut=700, focus=focus)
print(z_arr[1] - z_arr[2])
print((focus+dz_max*1e-3)*1e3)
# print(f" dz = {dz_max:.3f} mm \n dx = {dx_max:.3f} μm \n fwhm = {fwhm*1e6:.3f} μm \n I_max = {I_max:.1f} rel. un. \n div_ang = {div_ang}")
print(f" dz = {dz_max} mm \n dx = {dx_max} μm \n fwhm = {fwhm*1e6} μm \n I_max = {I_max} rel. un. \n div_ang = {div_ang}")
I_distr_max, I_max, dz_max, dx_max, fwhm, focus_image, z_arr, div_ang, x_cutted = crlm.focus_params(n_dots=1000, eps=3e-3, n_cut=700, focus=focus+dz_max*1e-3)

X, Y = np.meshgrid(x_cutted*1e6, z_arr*1000)
Z = focus_image

fig1,ax1 = plt.subplots(1, 1)
# fig1.set_figheight(15)
# fig1.set_figwidth(15)
colbar = plt.pcolormesh(X, Y, Z, cmap='inferno')
cbar = fig1.colorbar(colbar)
cbar.set_label('$I$, a.u.')

# plt.title("Intensity distribution at the focus (an ideal CRL)")
# plt.axvline(x = 0, color = 'g', alpha=0.8, linewidth=0.5)
# plt.plot([x_cutted[0]*1e6, x_cutted[-1]*1e6], [focus*1e3, focus*1e3], 'g', alpha=0.8, linewidth=0.5)
# plt.plot([(dx_max*1e-6 - fwhm/2)*1e6, (dx_max*1e-6 + fwhm/2)*1e6], [(dz_max*1e-3 + focus)*1e3, (dz_max*1e-3 + focus)*1e3], marker='|', color="blue", linewidth=0.5)
plt.xlabel("$x$, μm")
plt.ylabel("$z$, mm")
plt.savefig("1mkm", dpi=2400);

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np

# Создаем массив данных (можно использовать свои данные)
data = Z

# Создаем начальный график по индексу 0
fig, ax = plt.subplots()
l, = ax.plot(data[0])
ax.set_ylim(0, np.max(data))
ax.set_xlim(x_cutted[0], x_cutted[-1])
# Создаем слайдер
axcolor = 'lightgoldenrodyellow'
slider_ax = plt.axes([0.2, 0.02, 0.65, 0.03], facecolor=axcolor)
slider = Slider(slider_ax, 'Index', 0, len(data)-1, valinit=0, valstep=1)

# Функция обновления графика по индексу
def update(val):
    index = int(slider.val)
    l.set_ydata(data[index])
    l.set_xdata(x_cutted)
    ax.set_title("z = "+ f"{(z_arr[index] - focus)*1e3:.3f} мм")
    plt.draw()

slider.on_changed(update)

plt.show()

Тест от энергии E и числа линз N

In [ ]:
import cupy as cp
n_cut = 100
n_dots = 100

Copy_flag = True
arr_len = 2 if Copy_flag else 2*N_gl
phases = 2*np.pi*np.random.rand(arr_len)
w_s = (np.random.rand(arr_len)-0.5)*np.pi/180*0
s_s  = (np.random.rand(arr_len)-0.5)*2e-6*0




N_arr = np.arange(1, 100, 1)
E_arr = np.arange(10, 51, 5)
N_gl = 100

result_I = list()
result_I_m = list()
result_I_i = list()

for n in N_arr:
    N_gl = n

    p = od_1d.PointSource(z=100, En=15) 
    E_ps1 = p.E()/p.E()

    
    crl = od_1d.CRL(lam=p.lam, arr_start=E_ps1, z=0,\
                    R=6.25e-6, A=50e-6, d=2e-6, N_lens=N_gl, mol="Si", dens=2.33)

    crlm = od_1d.CRLm(lam=p.lam, arr_start=E_ps1, z=0, \
                    R=6.25e-6, A=50e-6, d=2e-6, N_lens=N_gl, mol="Si", dens=2.33, \
                    copy=Copy_flag, arr_phase=phases, arr_s=s_s, arr_w=w_s, b=1e-7, m=3e8)
    
    
    crlm_minus = od_1d.CRLm(lam=p.lam, arr_start=E_ps1, z=0, \
                    R=6.25e-6, A=50e-6, d=2e-6, N_lens=N_gl, mol="Si", dens=2.33, \
                    copy=Copy_flag, arr_phase=phases, arr_s=s_s, arr_w=w_s, b=-1e-7, m=3e8)
    

    # crlm.betta = 0
    crl.betta = 0 
    # crlm_minus.betta = 0

    focus = crlm.focus()

    eps_ = focus / 2
    
    I_distr_max, I_max, dz_max, dx_max, fwhm, focus_image, z_arr, div_ang, x_cutted = crlm.focus_params(n_dots=n_dots, eps=eps_, n_cut=n_cut)
    I_distr_max_i, I_max_i, dz_max_i, dx_max_i, fwhm_i, focus_image_i, z_arr_i, div_ang_i, x_cutted = crl.focus_params(n_dots=n_dots, eps=eps_, n_cut=n_cut)
    I_distr_max_m, I_max_m, dz_max_m, dx_max_m, fwhm_m, focus_image_m, z_arr_m, div_ang_m, x_cutted = crlm_minus.focus_params(n_dots=n_dots, eps=eps_, n_cut=n_cut)

    result_I_m.append([I_max_m, dz_max_m, fwhm_m, div_ang_m])
    result_I.append([I_max, dz_max, fwhm, div_ang])
    result_I_i.append([I_max_i, dz_max_i, fwhm_i, div_ang_i])
    # cp._default_memory_pool.free_all_blocks()

res_I = np.array(result_I).T
res_I_i = np.array(result_I_i).T
res_I_m = np.array(result_I_m).T

In [ ]:
import matplotlib.pyplot as plt
number = 0
x_plot = N_arr
# x_plot = E_arr
# plt.plot(x_plot, res_I[number] / np.log(x_plot), label="параллельный слой +0.1 мкм")
plt.plot(x_plot, res_I_i[number], label="идеальная линза")
# plt.plot(x_plot, res_I_m[number] / x_plot /np.log(x_plot) , label="параллельный слой -0.1 мкм")
plt.legend()
plt.yscale("log")
plt.xscale("log")
plt.grid()
# plt.xlabel("Энергия, КэВ")
# plt.plot(x_plot, np.exp(-2*crl.k*crl.betta*x_plot*crl.d))
print(crl.betta)
plt.xlabel("число линз")
plt.title(f"E = {p.En} кэв")
plt.ylabel(["пиковая интенсивность", "смещение фокуса вдоль оси z, мм", "полуширина пучка в фокусе, мкм", "угол расходимости"][number]);

In [ ]:
plt.plot(x_plot, np.log10(np.exp(x_plot**0.5)))
plt.xscale('log')
plt.yscale('log')